exp_DB_001_fromDeveicesToPerson.ipynb

This file transforms from Devices to Person

Daniel Bischoff, Jan 2024

In [131]:
import pandas as pd
import matplotlib.pyplot as plt
from tueplots import bundles
from tueplots.constants.color import rgb
import numpy as np

In [144]:
data_directory = '/Users/danielbischoff/Library/CloudStorage/OneDrive-UTCloud/Documents - Dateien von Philipp Langsteiner/Data_literacy_mensa/Mensa_data_combined/tuesday090124.csv'

# load csv file
data = pd.read_csv(data_directory)

/var/folders/z4/0m49679n1xd2nv8l_dt8zz880000gn/T/ipykernel_6659/3555762200.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_directory)


# Convert from Messages to Devices

In [145]:
data_uni_wlan = data.loc[(data['wlan.ssid'] == '656475726f616d') | (data['wlan.ssid'] == '55546775657374') | (data['wlan.ssid'] == '4775657374')]

src_devices = data_uni_wlan[['frame.time','wlan.sa']].rename(columns={
    'frame.time': 'time',
    'wlan.sa': 'adress',
    'wlan.ssid': 'ssid'
}).drop_duplicates()

dest_devices = data_uni_wlan[['frame.time','wlan.da']].rename(columns={
    'frame.time': 'time',
    'wlan.da': 'adress',
    'wlan.ssid': 'ssid'
}).drop_duplicates()

devices =  pd.concat([src_devices, dest_devices])

In [141]:
data_edurom = data.loc[(data['wlan.ssid'] == '656475726f616d')]
src_devices_edurom = data_edurom[['frame.time','wlan.sa']].rename(columns={
    'frame.time': 'time',
    'wlan.sa': 'adress',
    'wlan.ssid': 'ssid'
}).drop_duplicates()

dest_devices_edurom = data_edurom[['frame.time','wlan.da']].rename(columns={
    'frame.time': 'time',
    'wlan.da': 'adress',
    'wlan.ssid': 'ssid'
}).drop_duplicates()

devices_edurom =  pd.concat([src_devices_edurom, dest_devices_edurom])

In [143]:
# reasure plot
grouped_data_edurom = devices.groupby(devices_edurom['time'].dt.floor('Min')).size()

# Plot the sum of messages received in one minute
grouped_data_edurom.plot(kind='bar')
plt.xlabel('Time')
plt.ylabel('Number of Messages')
plt.title('Sum of Messages Received in One Minute')
interval_minutes = 15
tick_positions = np.arange(0, len(grouped_data_edurom.index), interval_minutes)
plt.xticks(tick_positions, [grouped_data_edurom.index[i] for i in tick_positions], rotation=45)
plt.show()

ValueError: cannot reindex on an axis with duplicate labels

In [146]:
# reasure plot
grouped_data = devices.groupby(devices['time'].dt.floor('Min')).size()

# Plot the sum of messages received in one minute
grouped_data.plot(kind='bar')
plt.xlabel('Time')
plt.ylabel('Number of Messages')
plt.title('Sum of Messages Received in One Minute')
interval_minutes = 15
tick_positions = np.arange(0, len(grouped_data.index), interval_minutes)
plt.xticks(tick_positions, [grouped_data.index[i] for i in tick_positions], rotation=45)
plt.show()

AttributeError: Can only use .dt accessor with datetimelike values

# Convert from Devices to People

In [120]:
# estimated time when all seats are occupied and there is no queue
number_of_seats = 532
# Create a datetime with CET timezone
fully_staffed_time = pd.Timestamp('2024-01-09 12:50:00+01:00', tz='CET')

# estimate devices per person
message_count_of_fully_staffed_time = devices['time'].eq(fully_staffed_time).sum()

average_device_number = message_count_of_fully_staffed_time / number_of_seats